In [ ]:
# https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://security.ubuntu.c

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [7]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

In [8]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

10319090
10316622
10316622


In [9]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
df1 = df.drop("marketplace") \
        .drop("product_category") \
        .drop("verified_purchase") \
        .drop("review_headline") \
        .drop("review_body")
df1.show(5)

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   28941543|R10014V9AVMQ73|0137150830|     854491944|The Greening of I...|          5|            1|          1|   N| 2014-09-18|
|      28650|R1004FYSAYGYNZ|8183860664|     188251931|Catching the Depa...|          4|            1|          1|   N| 2015-06-19|
|   35531538|R10086W0U9OHY1|1421695103|     828616333|Basset Hound Pupp...|          5|            0|          0|   N| 2012-12-28|
|   37127528|R100LDX2L70YXF|0439919045|     372719167|Littlest Pet Shop...|          5|            0|          0|   N| 2014-10-25|
|   18451121|R100OI6P955T8N|0764585924|     843831426|CliffsNotes on Sh...|        

In [11]:
df1.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)



In [14]:
from pyspark.sql.functions import col
from pyspark.sql.functions import to_date

df2=df1.withColumn('review_date', to_date('review_date', 'yyyy-MM-dd'))

In [15]:
df2.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)



In [16]:
df3=df2.withColumn("total_votes",df2.total_votes.cast('integer')) \
  .withColumn("customer_id",df2.customer_id.cast('integer')) \
  .withColumn("product_parent",df2.product_parent.cast('integer')) \
  .withColumn("star_rating",df2.star_rating.cast('integer')) \
  .withColumn("helpful_votes",df2.helpful_votes.cast('integer'))

In [17]:
df3.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)



In [19]:
df4 = df3.groupBy("customer_id").count()
customers=df4.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52942195|             2|
|   34912862|             3|
|   52013099|            31|
|   12325199|             1|
|   50785793|            59|
|   34302706|             1|
|   38047559|            19|
|   20366955|             1|
|   18989639|             7|
|   12294767|             4|
|   13238638|             1|
|   18950477|            46|
|    5334838|            26|
|   30655165|            10|
|   36925110|             6|
|   38086969|            15|
|   49657151|             2|
|   11914419|             7|
|   34608604|            80|
|   42177283|             1|
+-----------+--------------+
only showing top 20 rows



In [20]:
review_id_table = df3.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10014V9AVMQ73|   28941543|0137150830|     854491944| 2014-09-18|
|R1004FYSAYGYNZ|      28650|8183860664|     188251931| 2015-06-19|
|R10086W0U9OHY1|   35531538|1421695103|     828616333| 2012-12-28|
|R100LDX2L70YXF|   37127528|0439919045|     372719167| 2014-10-25|
|R100OI6P955T8N|   18451121|0764585924|     843831426| 2014-05-12|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [21]:
products = df3.select(["product_id", "product_title"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0137150830|The Greening of I...|
|8183860664|Catching the Depa...|
|1421695103|Basset Hound Pupp...|
|0439919045|Littlest Pet Shop...|
|0764585924|CliffsNotes on Sh...|
+----------+--------------------+
only showing top 5 rows



In [22]:
vine_table = df3.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10014V9AVMQ73|          5|            1|          1|   N|
|R1004FYSAYGYNZ|          4|            1|          1|   N|
|R10086W0U9OHY1|          5|            0|          0|   N|
|R100LDX2L70YXF|          5|            0|          0|   N|
|R100OI6P955T8N|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
config = {"user":"root",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)